Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [118]:
from random import random, seed
from itertools import product
from itertools import accumulate
import numpy as np
from matplotlib import pyplot as plt
from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [119]:
UNIVERSE_SIZE = 100000
NUM_SETS = 10000
DENSITY = 0.1

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [120]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [121]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [ ]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)

In [ ]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
valid(solution), cost(solution)

In [ ]:
solution = np.full(NUM_SETS, True)
SETS_ONE = SETS.astype(int)
history = [cost(solution)]

def improve1(solution, repetitions, steps):
    steps += 1
    index=-1
    max_num=0
    for i in range(NUM_SETS):
        if (solution[i]==True):
            occ = np.multiply(SETS_ONE[i], repetitions)
            if 1 not in occ:
                sum = occ.sum()
                if sum>max_num:
                    max_num=sum
                    index=i        
    if index!=-1:
        solution[index] = False
        history.append(cost(solution))
        repetitions -= SETS_ONE[index]
        ic(steps, cost(solution))
        improve1(solution, repetitions, steps)
    return

def improve2(solution, repetitions, steps):
    steps += 1
    index=np.full(100, -1)
    max_num=np.full(100, 0)
    for i in range(NUM_SETS):
        if (solution[i]==True):
            occ = np.multiply(SETS_ONE[i], repetitions)
            if 1 not in occ:
                sum = occ.sum()
                for x in range(100):
                    if sum>max_num[x]:
                        ref = 99
                        for y in range(99-x):
                            max_num[ref]=max_num[ref-1]
                            index[ref]=index[ref-1]
                            ref -= 1
                        max_num[x]=sum
                        index[x]=i
                        break  
    val = 0
    for n in range(100):
        if index[n]!=-1:
            prova = solution.copy()
            prova[index[n]] = False
            if (valid(prova)):
                val += 1
                solution[index[n]] = False
                repetitions -= SETS_ONE[index[n]]
    if index[0]!=-1:
        history.append(cost(solution))
        ic(steps, val, solution.sum(),  cost(solution))
        improve2(solution, repetitions, steps)
    return

def search():
    steps = 0
    rep = np.full(UNIVERSE_SIZE, 0)
    for s in range(NUM_SETS):
       rep += SETS_ONE[s]
    if (UNIVERSE_SIZE>10000):
       improve2(solution, rep, steps)
    else:
        improve1(solution, rep, steps)
    return

search()

plt.figure(figsize=(14, 8))
_ = plt.scatter(range(len(history)), history, marker=".")
